In [41]:
import pandas as pd

In [42]:
#Read the CSV and convert to pandas dataframe
df = pd.read_csv("CSV_To_Suppress.csv")

In [43]:
#Check the dataframe is created correctly
df

,Fin_Year,Public_private,DFJ_New,Region,Relationship_to_Child,Applicants,Unnamed: 6,Unnamed: 7
0,2015/16,Private Law,Birmingham DJF,MIDLANDS,Father,605,NaN,NaN
1,2015/16,Private Law,Birmingham DJF,MIDLANDS,Grandparent,74,NaN,NaN
2,2015/16,Private Law,Birmingham DJF,MIDLANDS,Mother,273,NaN,NaN
3,2015/16,Private Law,Birmingham DJF,MIDLANDS,Other,33,NaN,NaN
4,2015/16,Private Law,Birmingham DJF,MIDLANDS,Special Guardian,2,NaN,NaN
...,...,...,...,...,...,...,...,...
1897,2019/20,Public Law,Wrexham DFJ,WALES,Mother,5,NaN,NaN
1898,2019/20,Public Law,Wrexham DFJ,WALES,Other,2,NaN,NaN
1899,2019/20,Public Law,Wrexham DFJ,WALES,Unknown,18,NaN,NaN
1900,2019/20,Public Law,York DFJ,NORTH EAST,Other,1,NaN,NaN


In [44]:
#These are the variables which change in the regional totals
iterating_variables = ['Fin_Year','Public_private','Region','Relationship_to_Child']

In [45]:
#Creates lists of all the distinct values for each variable
Fin_Years = df.Fin_Year.unique()
Public_Private = df.Public_private.unique()
Region = df.Region.unique()
Relationship_to_Child = df.Relationship_to_Child.unique()

In [46]:
#Creates a list of all the unique combinations of regional totals 
combinations = [(a,b,c,d) for a in Fin_Years for b in Public_Private for c in Region for d in Relationship_to_Child]

In [47]:
len(combinations)

490

In [48]:
list_to_secondary_suppress = []
for i in range(len(combinations)): #Runs through each combination
    
    #For each combination, get a list of all the Applicants totals
    iteration_of_applicants = df.loc[(df[iterating_variables[0]] == combinations[i][0]) &
               (df[iterating_variables[1]] == combinations[i][1])&
               (df[iterating_variables[2]] == combinations[i][2])& 
               (df[iterating_variables[3]] == combinations[i][3])].Applicants
    
    iteration_app_total = sum(iteration_of_applicants) #Calculate total applicants for this combination
    
    if iteration_app_total > 5: #If regional total isn't suppressed    
        suppressed_apps = df.loc[(df[iterating_variables[0]] == combinations[i][0]) &
                   (df[iterating_variables[1]] == combinations[i][1])&
                   (df[iterating_variables[2]] == combinations[i][2])& 
                   (df[iterating_variables[3]] == combinations[i][3])
                    & (df['Applicants']<=5)& (df['Applicants']>=1)].Applicants 
                    #gets a list of values in the range to be suppressed
        
        #If there is only 1 value to be primary suppressed, or if values are all 1s or all 5s then flag combination as needing secondary suppression
        if (len(suppressed_apps)==1 or ((len(suppressed_apps)==sum(suppressed_apps) or 5*len(suppressed_apps)==sum(suppressed_apps)) and len(suppressed_apps)>0)):
            list_to_secondary_suppress.append(combinations[i])

In [49]:
#check the list produced
len(list_to_secondary_suppress)

39

In [50]:
for index,row in df.iterrows():
    #primary suppress any values between 1 or 5
    if row['Applicants']<=5 and row['Applicants']>=1:
        df.at[index,'Applicants']='*'

In [51]:
for i in range(len(list_to_secondary_suppress)):
    
    #Iterates through the secondary suppression flag list and creates a list of applicant totals for every combination (ignoring already suppressed values)
    iteration_of_applicants = df.loc[(df[iterating_variables[0]] == list_to_secondary_suppress[i][0]) &
               (df[iterating_variables[1]] == list_to_secondary_suppress[i][1])&
               (df[iterating_variables[2]] == list_to_secondary_suppress[i][2])& 
               (df[iterating_variables[3]] == list_to_secondary_suppress[i][3])&
                (df['Applicants']!='*')]
    
    #Finds the index of the lowest value not primary suppressed
    min_apps = -1
    for index in iteration_of_applicants.index:
        if min_apps==-1:
            min_apps = df.at[index,'Applicants']
            min_index = index
        else:
            if df.at[index,'Applicants']<min_apps:
                min_apps=df.at[index,'Applicants']
                min_index=index
    #secondary suppresses the minimum value
    df.at[min_index,'Applicants']='*'


In [52]:
df

,Fin_Year,Public_private,DFJ_New,Region,Relationship_to_Child,Applicants,Unnamed: 6,Unnamed: 7
0,2015/16,Private Law,Birmingham DJF,MIDLANDS,Father,605,NaN,NaN
1,2015/16,Private Law,Birmingham DJF,MIDLANDS,Grandparent,74,NaN,NaN
2,2015/16,Private Law,Birmingham DJF,MIDLANDS,Mother,273,NaN,NaN
3,2015/16,Private Law,Birmingham DJF,MIDLANDS,Other,33,NaN,NaN
4,2015/16,Private Law,Birmingham DJF,MIDLANDS,Special Guardian,*,NaN,NaN
...,...,...,...,...,...,...,...,...
1897,2019/20,Public Law,Wrexham DFJ,WALES,Mother,*,NaN,NaN
1898,2019/20,Public Law,Wrexham DFJ,WALES,Other,*,NaN,NaN
1899,2019/20,Public Law,Wrexham DFJ,WALES,Unknown,18,NaN,NaN
1900,2019/20,Public Law,York DFJ,NORTH EAST,Other,*,NaN,NaN
